<a href="https://colab.research.google.com/github/minhyeokpark/pdm06/blob/main/notebook/chap07_MLP_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP: Keras

## mini-batch 구현 

In [1]:
import numpy as np
import tensorflow as tf

# 데이터를 학습 데이터와 테스트 데이터로 나누어준다. 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 

print(x_train.shape) # train 데이터는 28x28 영상이 60000개 있음. 3차원 행렬 
print(y_train.shape) # y_train 데이터는 (60000,)
print(y_train[:10]) # 5,0,4,1,.... 숫자 그림의 1번째 ~10번째 까지 정답이 출력 
#x_train.shape[0] = 60000  | [1] = 28 

print(x_test.shape , y_test.shape) #10000개 의 28x28 영상 |   
print(y_test[:10]) # 7,2,1,0... 
#x_test.shape[0] = 10000 

data_size = x_train.shape[0] # 60000
batch_size = 12	# 배치 크기

selected = np.random.choice(data_size, batch_size) #60000개 데이터 중 12개 정수 선택. 
#random seed 를 고정하지 않다면 랜덤으로 12개 값이 선별됌. 
print(selected)
x_batch = x_train[selected]
y_batch = y_train[selected]
x_batch.shape, y_batch.shape #28x28 영상 12개 | 12 개 숫자 


11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4 1 9 2 1 3 1 4]
(10000, 28, 28) (10000,)
[7 2 1 0 4 1 4 9 5 9]
[38731 44384 27384 57332 11221  6400  4867 49329 39872 27813 43151 16628]


((12, 28, 28), (12,))

## mini-batch: XOR using MLP 

In [2]:
import numpy as np
# XOR solution by MLP

# 시그모이드 함수
def actf(x):
	return 1/(1+np.exp(-x))

# 시그모이드 함수의 미분치
def actf_deriv(x):
	    return x*(1-x)

# 입력유닛의 개수, 은닉유닛의 개수, 출력유닛의 개수
inputs, hiddens, outputs = 2, 2, 1
learning_rate = 0.5

# 훈련 입력과 출력
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
T = np.array([[0], [1], [1], [0]]) #XOR

# 가중치를 –1.0에서 1.0 사이의 난수로 초기화한다.
W1 = 2*np.random.random((inputs, hiddens))-1 #input = 2 hidden = 2 | w1 = -1~1 실수로 무작위로 만듬 
W2 = 2*np.random.random((hiddens, outputs))-1  #w2 = 2x2 으로 -1~1 까지 만들어줌. 
B1 = np.zeros(hiddens)   #2x1
B2 = np.zeros(outputs)   #1x1

# 순방향 전파 계산
def predict(x):
        layer0 = x			# 입력을 layer0에 대입한다. 
        Z1 = np.dot(layer0, W1)+B1	# 행렬의 곱을 계산한다. 
        layer1 = actf(Z1)		# 활성화 함수를 적용한다. 
        Z2 = np.dot(layer1, W2)+B2	# 행렬의 곱을 계산한다. 
        layer2 = actf(Z2)		# 활성화 함수를 적용한다. 
        return layer0, layer1, layer2
    
# 역방향 전파 계산
def fit():
    global W1, W2, B1, B2
    for i in range(60000):
            layer0, layer1, layer2 = predict(X) # input-batch-size = 4
            layer2_error = layer2-T

            layer2_delta = layer2_error*actf_deriv(layer2)
            layer1_error = np.dot(layer2_delta, W2.T)
            layer1_delta = layer1_error*actf_deriv(layer1)
            
            W2 += -learning_rate*np.dot(layer1.T, layer2_delta)/4.0 # 4개의 입력에 대한 평균 기울기
            W1 += -learning_rate*np.dot(layer0.T, layer1_delta)/4.0
            B2 += -learning_rate*np.sum(layer2_delta, axis=0)/4.0
            B1 += -learning_rate*np.sum(layer1_delta, axis=0)/4.0

def test(): # fit 로 가중치를 훈련 한 후 검증하는 단계  - global w1 w2 b1 b2 의 9개의 가중치를 6만번 훈련한 가중치로. 
    for x, y in zip(X, T):
        x = np.reshape(x, (1, -1))		# 하나여도 2차원 형태이어야 한다.
        layer0, layer1, layer2 = predict(x)
        print(x, y, layer2)

fit()
test()
W1,W2
B1,B2


[[0 0]] [0] [[0.01182247]]
[[0 1]] [1] [[0.49975666]]
[[1 0]] [1] [[0.98601491]]
[[1 1]] [0] [[0.50043355]]


(array([ 2.05690406, -0.9704361 ]), array([-1.0236828]))